# ✂️ Model Pruning From Scratch: Sparsity and Significance

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/pruning_demo.ipynb)

## 📖 The Theory: Magnitude-based Pruning

Neural networks are often **over-parameterized**, meaning many weights contribute very little to the final result. Pruning is the process of zeroing out these insignificant weights.

### L1 Magnitude Pruning
The most common heuristic is that weights with the smallest absolute values (L1 norm) are the least important. By setting them to zero, we create a **sparse matrix**.

### Masking
In practice, we don't just delete the weight. We maintain a binary **mask** $M$ of the same shape as $W$. The effective weight used in the model is then:

$$W_{pruned} = W \odot M$$

---

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel

def manual_l1_pruning(weight, amount=0.3):
    """
    Implementation of L1 Unstructured Pruning from scratch.
    weight: Tensor
    amount: Fraction of weights to prune (0.0 to 1.0)
    """
    # 1. Flatten weight and compute L1 magnitude
    w_abs = torch.abs(weight).flatten()
    
    # 2. Find the threshold value for the bottom % of weights
    k = int(amount * w_abs.numel())
    if k == 0: return weight, torch.ones_like(weight)
    
    threshold = torch.kthvalue(w_abs, k).values
    
    # 3. Create Binary Mask (1 if > threshold, 0 otherwise)
    mask = (torch.abs(weight) > threshold).float()
    
    # 4. Apply Mask
    return weight * mask, mask

# Test on GPT-2 Layer
model = GPT2LMHeadModel.from_pretrained("gpt2")
layer_weight = model.transformer.h[0].attn.c_attn.weight.data.clone()

pruned_w, mask = manual_l1_pruning(layer_weight, amount=0.5)

print(f"Original Params: {layer_weight.numel()}")
print(f"Zeroed Params:   {torch.sum(mask == 0).item()}")
print(f"Sparsity:        {100. * torch.sum(mask == 0).item() / layer_weight.numel():.2f}%")

## 🔢 Worked Example with numbers

A step-by-step trace of `manual_l1_pruning` with a tiny weight matrix so you can verify every value by hand.

- Weight matrix **2 × 5**, pruning **30 %** of weights (`amount = 0.3`)
- 10 total weights → **k = 3** weights will be zeroed out

In [ ]:
import torch

# 2×5 weight matrix (10 elements total)
weight = torch.tensor([[ 0.9,  0.2,  0.6,  0.1,  0.5],
                        [ 0.8,  0.3,  0.7, 0.05,  0.4]])   # shape [2, 5]

amount = 0.3   # prune the 30 % smallest-magnitude weights

# ── Step 1 : Flatten and compute |weight| ──────────────────────────────────
w_abs = torch.abs(weight).flatten()
# w_abs  =>  [0.9, 0.2, 0.6, 0.1, 0.5, 0.8, 0.3, 0.7, 0.05, 0.4]

# ── Step 2 : Find the pruning threshold ────────────────────────────────────
k = int(amount * w_abs.numel())   # => int(0.3 × 10) = 3
# Sorted ascending: [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#                     ^1    ^2   ^3   ← 3rd-smallest is the threshold
threshold = torch.kthvalue(w_abs, k).values   # => 0.2

# ── Step 3 : Build the binary mask  (1 = keep, 0 = prune) ─────────────────
# |weight|:   [[0.9, 0.2, 0.6, 0.1, 0.5],  [0.8, 0.3, 0.7, 0.05, 0.4]]
# > 0.2 :     [[  1,   0,   1,   0,   1],   [  1,   1,   1,    0,   1]]
mask = (torch.abs(weight) > threshold).float()
# mask        =>  [[1., 0., 1., 0., 1.],
#                  [1., 1., 1., 0., 1.]]

# ── Step 4 : Apply mask  (element-wise multiply) ──────────────────────────
pruned_weight = weight * mask
# weight × mask:
#   [[ 0.9×1,  0.2×0,  0.6×1,  0.1×0,  0.5×1],
#    [ 0.8×1,  0.3×1,  0.7×1, 0.05×0,  0.4×1]]
# pruned_weight  =>  [[ 0.9,  0.0,  0.6,  0.0,  0.5],
#                     [ 0.8,  0.3,  0.7,  0.0,  0.4]]

zeroed   = int(torch.sum(mask == 0).item())        # => 3  (0.2, 0.1, 0.05 were zeroed)
sparsity = 100.0 * zeroed / weight.numel()         # => 3 / 10 × 100 = 30.0 %

print(f"k (weights to prune) : {k}")               # => 3
print(f"Threshold            : {threshold:.2f}")   # => 0.20
print(f"Mask :\n{mask}")
# => [[1., 0., 1., 0., 1.],
#     [1., 1., 1., 0., 1.]]
print(f"Pruned weight :\n{pruned_weight}")
# => [[ 0.9000,  0.0000,  0.6000,  0.0000,  0.5000],
#     [ 0.8000,  0.3000,  0.7000,  0.0000,  0.4000]]
print(f"Zeroed params        : {zeroed} / {weight.numel()}")   # => 3 / 10
print(f"Sparsity             : {sparsity:.1f}%")               # => 30.0%

## 🧱 Structured vs. Unstructured
While the code above prunes individual weights (**Unstructured**), modern hardware often prefers pruning entire rows or columns (**Structured**) to achieve real-world latency gains.